# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from scipy.stats import uniform

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split       

from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from numpy.random import RandomState

import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_0.info()
data_1.info()
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [4]:
data_2.head(3)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


*id* — уникальный идентификатор скважины;

*f0, f1, f2* — три признака точек;

*product* — объём запасов в скважине (тыс. баррелей).

Представленные в данных признаки уже отмасштабированны.

Проверка на  явные дубликаты дубликаты.

In [5]:
print(data_0.duplicated().sum())
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())

0
0
0


Проверим признаки на корреляцию.

In [7]:
display(data_0.corr().style.background_gradient('PuBuGn'))
display(data_1.corr().style.background_gradient('PuBuGn'))
display(data_2.corr().style.background_gradient('PuBuGn'))

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Корреляции признаков меджу собой не наблюдается. Значительная линейная корреляция признака f2 и целевого признака во втором регионе.

Разделим выборки на тренировочные и валидационные для каждого из датасетов.

In [8]:
# для первого региона
features_0 = data_0.drop(columns=['id','product'])
target_0 = data_0['product']

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0,target_0,
                                                                                      test_size=0.25 ,random_state=12345)

# для второго региона
features_1 = data_1.drop(columns=['id','product'])
target_1 = data_1['product']

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1,target_1,
                                                                                      test_size=0.25 ,random_state=12345)

# для третьего региона
features_2 = data_2.drop(columns=['id','product'])
target_2 = data_2['product']

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2,target_2,
                                                                                      test_size=0.25 ,random_state=12345)

## Обучение и проверка модели

In [9]:
results = pd.DataFrame(columns=['R2','RMSE','MAE','Predicted Product'])

In [10]:
def get_metrics(model,features_x,target_x,predictions_x):
    r2 = r2_score(target_x,predictions_x)
    rmse = mean_squared_error(target_x,predictions_x) ** 0.5
    mae = mean_absolute_error(target_x,predictions_x)
    
    return r2,rmse,mae, predictions_x.mean()
    #print(f'R2: {r2:.3f}\nRMSE: {rmse:.3f}\nMAE: {mae:.3f}\nСреднее количество сырья: {predictions_x.mean():.3f}')

In [11]:
model_0 = LinearRegression()
model_0.fit(features_train_0,target_train_0)
predictions_0 = model_0.predict(features_valid_0)

results.loc['Region 1',:] = get_metrics(model_0,features_valid_0,target_valid_0,predictions_0)

In [12]:
model_1 = LinearRegression()
model_1.fit(features_train_1,target_train_1)
predictions_1 = model_1.predict(features_valid_1)

results.loc['Region 2',:] =get_metrics(model_1,features_valid_1,target_valid_1,predictions_1)

In [13]:
model_2 = LinearRegression()
model_2.fit(features_train_2,target_train_2)
predictions_2 = model_2.predict(features_valid_2)

results.loc['Region 3',:] =get_metrics(model_2,features_valid_2,target_valid_2,predictions_2)

In [14]:
results['Real Product'] = [data_0['product'].mean(),data_1['product'].mean(),data_2['product'].mean()]
results

,R2,RMSE,MAE,Predicted Product,Real Product
Region 1,0.279943,37.579422,30.919601,92.592568,92.500
Region 2,0.999623,0.893099,0.718766,68.728547,68.825
Region 3,0.205248,40.029709,32.792652,94.965046,95.000


**Вывод:** Наилушие метрики показала модель линейной регрессии, построенная по данным второго региона.

## Подготовка к расчёту прибыли

Из условия задачи, бюджет на разработку скважин в регионе составляет 10 млрд рублей. Количество скважин, которые необходимо пробурить -  200. Таким образом можем приблизительно оценить стоимость одной скважины.  №№№№№№

In [15]:
region_budget = 10_000_000_000 # бюджет на разработку месторождений в регионе
well_count = 200               # количество скважин
barrel_price = 450_000         # доход за тысячу баррелей нефти


well_price = region_budget / well_count  # цена разработки одной скважины

barrel_count = well_price / barrel_price # количество сырья для окупаемости скважины, тыс. баррелей

print(f'Цена разработки одной скважины, руб: {well_price:.0f}\nКоличество сырья для окупаемости скважины, тыс. баррелей: {barrel_count:.4f}')

Цена разработки одной скважины, руб: 50000000
Количество сырья для окупаемости скважины, тыс. баррелей: 111.1111


In [16]:
def mean_compare(data,barrel_count):
    return (f'Средний запас в регионе больше {barrel_count:.4f}' if data['product'].mean() >= barrel_count 
      else f'Средний запас в регионе меньше {barrel_count:.4f}')

In [17]:
mean_compare(data_0,barrel_count)

'Средний запас в регионе меньше 111.1111'

In [18]:
mean_compare(data_1,barrel_count)

'Средний запас в регионе меньше 111.1111'

In [19]:
mean_compare(data_2,barrel_count)

'Средний запас в регионе меньше 111.1111'

**Вывод:** Средний запас по всем регионам меньше необходимого для окупаемости одной скважены. Такой подход не позволяет достоверно оценить риски, поэтому воспользумся методом bootstrap.

## Расчёт прибыли и рисков 

In [20]:
RND_ST = RandomState(12345)

Функция для вычисления прибыли.

In [25]:
def estimate_profit(products,barrel_price_x,region_budget_x):
    return products.sum() * barrel_price_x - region_budget_x 

 Функция *Bootstrap*

In [33]:
def boostraping(target,predictions_x,barrel_price_x,region_budget_x):
    
    res = []
    for _ in range(1000):
        subsample  = pd.Series(predictions_x).sample(n=500,random_state=RND_ST).sort_values(ascending=False).head(200)
        
        res.append(estimate_profit(target.iloc[subsample.index],barrel_price_x,region_budget_x ))
        
    res = pd.Series(res)
    return  len(res[res <= 0]),(len(res[res <= 0]) / len(res)) *100 , res.mean(), res.quantile(0.975) , res.quantile(0.025), res.min()

In [32]:
res = pd.DataFrame(columns=['Кол-во неприбыльных скважин','Доля неприбыльных скважин,%','Средняя прибыль,млн','97,5 квантиль','2,5 квантиль','min'])

In [34]:
res.loc['Region 1',:] = boostraping(target_valid_0,predictions_0,barrel_price,region_budget)
res.loc['Region 2',:] = boostraping(target_valid_1,predictions_1,barrel_price,region_budget)
res.loc['Region 3',:] = boostraping(target_valid_2,predictions_2,barrel_price,region_budget)

res.loc[:,'Средняя прибыль,млн':'min'] = res.loc[:,'Средняя прибыль,млн':'min'] / 1_000_000 

res

,Кол-во неприбыльных скважин,"Доля неприбыльных скважин,%","Средняя прибыль,млн","97,5 квантиль","2,5 квантиль",min
Region 1,72,7.2,375.564158,912.125446,-133.909614,-557.755629
Region 2,13,1.3,457.892565,872.29914,68.89033,-213.727052
Region 3,77,7.7,393.957806,919.885438,-145.234944,-474.527283


**Вывод:** Предпочтительным для разработки месторождений является Регион 2. Доля неприбыльных скважин не превышает 2.5%, показатели средней прибыли по региону выше, чем у двух других. Нижней границей 95% доверительного интервала является положительное число, что исключает возможность понести убытки.

## Общий вывод

Изучены и подготовлены данные по 3-м регионам для разработки нефтяных меторождений.

Обучены модели линейной регрессии, проведены оценки метрик.

Определен минимальный объем сырья для окупаемости разработки одной скважены.

Проведена оценка средней прибыли, 95% доверительнго интервала и доли неприбыльных скважин для каждого из регионов.

Для разработки месторождений был рекомендован регион 2, в связи с наименьшими рисками получения убытков.